In [38]:
import  os
import json
import pandas as pd
import traceback


In [39]:
# Install the langchain package
#%pip install langchain_community

from langchain.chat_models import ChatOpenAI


In [51]:
from dotenv import load_dotenv
load_dotenv()

True

In [75]:
KEY=os.getenv("OPENAI_API_KEY")


In [76]:
print(KEY)

sk-efgh5678abcd1234efgh5678abcd1234efgh5678


In [77]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo",temperature=0.5, max_tokens=100)

In [78]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000025D3EF38D30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000025D3EF3C700>, temperature=0.5, openai_api_key='sk-efgh5678abcd1234efgh5678abcd1234efgh5678', openai_proxy='', max_tokens=100)

In [79]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [82]:
RESPONSE_JSON = {
    "1": {
        "mcq": "Multiple choice question",
         "options": {
            "a": "Choice here",
            "b": "Choice here",
            "c": "Choice here",
            "d": "Choice here",
        },
        "correct": "Correct Answer"
    },
    "2": {
        "mcq": "Multiple choice question",
        "options": {
            "a": "Choice here",
            "b": "Choice here",
            "c": "Choice here",
            "d": "Choice here",
        },
        "correct": "Correct Answer"
    },
    "3": {
        "mcq": "Multiple choice question",
        "options": {
            "a": "Choice here",
            "b": "Choice here",
            "c": "Choice here",
            "d": "Choice here",
        },
        "correct": "Correct Answer"
    },
}

In [83]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
crete a quiz of {number} multiple choice questions for {subject} students in {tone}.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON beow and use it as a guide. \
Ensure to make {number} MCQs
### RESPNSE_JSON
{response_json}
"""

In [84]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "numer", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [85]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="quiz",
    verbose=True    
)

In [86]:
TEMPLATES2="""
You are an expert english grammarian and writer. Given a Multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity if the quiz is not at per with the cognitive analytical abilities of the students, \
update the quiz questios which need to be changed change the tone of such that it perfectly fits the student.
Quiz_MCQs:
{quiz}

Check from an expert English witer of the above quiz:
"""

In [87]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [88]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [89]:
from langchain.chains import SequentialChain

generate_evaluate_chain = SequentialChain(
	chains=[quiz_chain, review_chain],
	input_variables=["text", "number", "subject", "tone", "response_json"],
	output_variables=["quiz", "review"],
	verbose=True
)

In [90]:
file_path=r"C:\Malatesh\Learning\Project1\data.txt"

In [91]:
file_path

'C:\\Malatesh\\Learning\\Project1\\data.txt'

In [92]:
with open(file_path, "r") as file:
    text = file.read()

In [93]:
text

'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contribu

In [94]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct": "Correct Answer"}, "2": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct": "Correct Answer"}, "3": {"mcq": "Multiple choice question", "options": {"a": "Choice here", "b": "Choice here", "c": "Choice here", "d": "Choice here"}, "correct": "Correct Answer"}}'

In [95]:
#number=5
SUBJECT = "Machine Learning"
TONE = "Formal"

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":text, 
            "number":5, 
            "subject":SUBJECT, 
            "tone":TONE, 
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-efgh5*******************************5678. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}